# Airtable Reference Manager Tools
### Analysis and more, with airtable+pandas

In [59]:
# Initialize The API classes
import airtable.api
import pandas
import pprint
api = airtable.api.AirtableAPI()
refapi = airtable.api.ReferencesAPI()
authors_api = airtable.api.AuthorsAPI()

rdf = refapi.to_df()
author_df = authors_api.to_df()

# Simple Lookups

In [44]:
# Get all papers authored by an author
selected_author = author_df.sample()
print "Papers Authored by {}".format(selected_author['Author Name'])
print rdf.loc[selected_author['Papers Authored'][0]][['Title', 'Year']]

Papers Authored by recnGiiDyErc3FXrf    Jurgen Schmidhuber
Name: Author Name, dtype: object
                                                               Title  Year
recMLeRzdy0XRWCMN  A first look at music composition using lstm r...  2002


In [55]:
# Get all authors from a paper authored by an author
selected_paper = rdf.sample()
print "Authors from paper {}".format(selected_paper['Title'])
print author_df.loc[selected_paper['Authors'][0]][['Author Name']]

Authors from paper recdvTIvsjGt0Mdr5    Recurrent neural networks for music computation
Name: Title, dtype: object
                        Author Name
recBIc6L00JkEgSlI  Judy A. Franklin


# Stats

In [182]:
# Number of papers from each year
rdf['Year'].value_counts().sort_index()

1997    2
2001    1
2002    2
2005    1
2006    2
2007    1
2011    1
2012    1
2013    1
2014    2
2015    2
2016    1
Name: Year, dtype: int64

In [183]:
# Number of papers published by each author
n_authored = pandas.Series(author_df['Papers Authored'].dropna().map(lambda x: len(x)), name='N Authored')
print author_df.join(n_authored)[['Author Name', 'N Authored']].to_string(index=False)

            Author Name  N Authored
               S Keshav           1
         Paolo Frasconi           1
              DP Kingma           1
       Judy A. Franklin           1
   Abdel-rahman Mohamed           1
         Ilya Sutskever           3
              mark sand           1
 Ruslan R Salakhutdinov           1
       Wojciech Zaremba           1
           Mark Sandler         NaN
      Kuldip K. Paliwal           1
     Christian Szegedey           1
           Keunwoo Choi           1
          Oriol Vinyals           1
        Sepp Hochreiter           2
          Mike Schuster           1
          James Martens           1
      Andrew J. Lambert           1
            Alex Graves           3
     Jurgen Schmidhuber           5
         George Fazekas           1
          Tillman Weyde           1
          Yoshua Bengio           1
       Newton Armstrong           1
        Geoffrey Hinton           4
   Nicol N. Schraudolph           1
     Jurgen Schmidhuber     

# Author Graphs
Now a little fun with author connections

In [193]:
# Get All 1-step connections to an author
selected_author = author_df.sample()
print "Searching for author: {}".format(selected_author['Author Name'][0])
# Get all papers by that author
papers = rdf.loc[selected_author['Papers Authored'][0]]
# Get All authors from those papers, minus the selected author.
author_list = []
for index, paper in papers.iterrows():
    author_list.append(paper['Authors'][0])
shared_authors = set(author_list) - set(selected_author.index)

print
print "This author shares papers with:"
print author_df.loc[shared_authors][['Author Name']].to_string(index=False)

Searching for author: Geoffrey Hinton

This author shares papers with:
    Author Name
    Alex Graves
 Ilya Sutskever


# BibTex
## Creating a bibtex markup.

Since you ahve included the raw BibTex Markup in the reference table, making your bibtex file is now easy!

In [168]:
import datetime
def bibtex_from_df(df):
    """Create a bibtex string from"""
    bibtex_str = ""
    bibtex_str += "%% Bibtex file generated from Airtable Reference Manager\n"
    bibtex_str += "%% Generated at {}\n\n".format(datetime.datetime.now())
    
    bibtex_str += ",\n".join(df['BibTex Reference'].dropna().map(lambda x: x.strip('\n')))
    return bibtex_str

def write_bibtex(bibtex, filename):
    with open(filename, 'w') as fh:
        fh.write(bibte)

# Write your whole references file to a bibtex string
print bibtex_from_df(rdf)

%% Bibtex file generated from Airtable Reference Manager
%% Generated at 2016-06-12 12:02:03.344259

@article{keshav2007read,
  title={How to read a paper},
  author={Keshav, S},
  journal={ACM SIGCOMM Computer Communication Review},
  volume={37},
  number={3},
  pages={83--84},
  year={2007},
  publisher={ACM}
},
@inproceedings{sutskever2011generating,
  title={Generating text with recurrent neural networks},
  author={Sutskever, Ilya and Martens, James and Hinton, Geoffrey E},
  booktitle={Proceedings of the 28th International Conference on Machine Learning (ICML-11)},
  pages={1017--1024},
  year={2011}
},
@article{choi2016text,
  title={Text-based LSTM networks for Automatic Music Composition},
  author={Choi, Keunwoo and Fazekas, George and Sandler, Mark},
  journal={arXiv preprint arXiv:1604.05358},
  year={2016}
},
@article{eck2002first,
  title={A first look at music composition using lstm recurrent neural networks},
  author={Eck, Douglas and Schmidhuber, Juergen},
  journal=

In [169]:
# ... Or choose a subset!
# Write your whole references file to a bibtex string
print bibtex_from_df(rdf.sample(5))

%% Bibtex file generated from Airtable Reference Manager
%% Generated at 2016-06-12 12:02:10.440437

@article{keshav2007read,
  title={How to read a paper},
  author={Keshav, S},
  journal={ACM SIGCOMM Computer Communication Review},
  volume={37},
  number={3},
  pages={83--84},
  year={2007},
  publisher={ACM}
},
@article{hinton2006reducing,
  title={Reducing the dimensionality of data with neural networks},
  author={Hinton, Geoffrey E and Salakhutdinov, Ruslan R},
  journal={Science},
  volume={313},
  number={5786},
  pages={504--507},
  year={2006},
  publisher={American Association for the Advancement of Science}
},
@article{eck2002first,
  title={A first look at music composition using lstm recurrent neural networks},
  author={Eck, Douglas and Schmidhuber, Juergen},
  journal={Istituto Dalle Molle Di Studi Sull Intelligenza Artificiale},
  volume={103},
  year={2002}
},
@article{graves2005framewise,
  title={Framewise phoneme classification with bidirectional LSTM and other ne